# Prerequisites

In [1]:
import json
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import transformers

# package in . directory
from bichoice import utils
from baseline.model import BertForClassification
from baseline.data_processor import DataProcessor

In [28]:
assert transformers.__version__ == '4.1.1'

# declare a namespace
D = utils.GlobalSettings({
        'DATADIR': './data/',
        # select checkpoint-7
        'MODELDIR': './outputs/baseline_output/checkpoint-7/',
    })

# load training parameters
argD = utils.GlobalSettings(
    torch.load(os.path.join(D.MODELDIR, 'training_args.bin')))
print('this model is trained with following hyper parameters:')
print(str(argD))

this model is trained with following hyper parameters:
{
    "model_name": "bert-base-chinese",
    "data_dir": "../data/c3_data/",
    "max_length": 512,
    "num_train_epochs": 8,
    "train_batch_size": 4,
    "eval_batch_size": 16,
    "learning_rate": 2e-05,
    "gradient_accumulation_steps": 6,
    "num_warmup_steps": 500,
    "max_grad_norm": 1.0,
    "n_class": 4,
    "fp16": 1,
    "seed": 42,
    "save_dir": "../output/checkpoints/",
    "output_dir": "../output/outputs/",
    "log_dir": "../output/logs"
}


# Show Some Examples

In [29]:
processor = DataProcessor(D.DATADIR)
tokenizer = transformers.BertTokenizer.from_pretrained(argD.model_name)
train = processor.dataset[0]
# select a literal example from train set
train_e = random.choice(train)

In [18]:
def show_baseline_example(e):
    '''show all info of a single `baseline.InputExample` object`'''
    print('text_a:')
    print('    ', e.text_a)
    print('text_b:')
    print('    ', e.text_b)
    print('text_c:')
    print('    ', e.text_c)
    print('label:', e.label)
    print('guid:', e.guid)

In [19]:
# create several baseline examples from `train_e`
train_b_e = processor._create_examples([train_e], set_type='train')
for i, e in enumerate(train_b_e):
    print('-----EXAMPLE{}-----'.format(i+1))
    show_baseline_example(e)

-----EXAMPLE1-----
text_a:
     在很久很久以前，乌龟和兔子之间发生了争论，它们都说自己跑得比对方快。于是，它们决定通过比赛来一决胜负。确定了路线之后它们就开始跑了起来。兔子一个箭步冲到了前面，并且一路领先。它看到乌龟被远远抛在了后面，兔子觉得，自己应该先在树下休息一会儿。于是，它在树下坐了下来，并且很快睡着了。乌龟慢慢地超过了它，并且完成了整个赛程，开心地当上了冠军。后来，兔子醒了过来，发现自己输了。
text_b:
     它们之间发生了争执
text_c:
     乌龟和兔子为什么要进行比赛?
label: 2
guid: train-0-0
-----EXAMPLE2-----
text_a:
     在很久很久以前，乌龟和兔子之间发生了争论，它们都说自己跑得比对方快。于是，它们决定通过比赛来一决胜负。确定了路线之后它们就开始跑了起来。兔子一个箭步冲到了前面，并且一路领先。它看到乌龟被远远抛在了后面，兔子觉得，自己应该先在树下休息一会儿。于是，它在树下坐了下来，并且很快睡着了。乌龟慢慢地超过了它，并且完成了整个赛程，开心地当上了冠军。后来，兔子醒了过来，发现自己输了。
text_b:
     它们的关系很不好
text_c:
     乌龟和兔子为什么要进行比赛?
label: 2
guid: train-0-1
-----EXAMPLE3-----
text_a:
     在很久很久以前，乌龟和兔子之间发生了争论，它们都说自己跑得比对方快。于是，它们决定通过比赛来一决胜负。确定了路线之后它们就开始跑了起来。兔子一个箭步冲到了前面，并且一路领先。它看到乌龟被远远抛在了后面，兔子觉得，自己应该先在树下休息一会儿。于是，它在树下坐了下来，并且很快睡着了。乌龟慢慢地超过了它，并且完成了整个赛程，开心地当上了冠军。后来，兔子醒了过来，发现自己输了。
text_b:
     它们都认为自己跑得快
text_c:
     乌龟和兔子为什么要进行比赛?
label: 2
guid: train-0-2
-----EXAMPLE4-----
text_a:
     在很久很久以前，乌龟和兔子之间发生了争论，它们都说自己跑得比对方快。于是，它们决定通过比赛来一决胜负。确定了路线之后它们就开始跑了起来。兔子一个箭步冲到了前面，并且一

# Tokenizing Examples

In [46]:
def show_baseline_features(f):
    '''show info of a single `baseline.InputFeatures` object'''
    print('-----FIRST TOKEN SEQUENCE-----')
    input_mask = np.asarray(f.input_mask)
    input_ids = np.asarray(f.input_ids)[input_mask==1]
    segment_ids = np.asarray(f.segment_ids)[input_mask==1]
    first_sent = tokenizer.convert_ids_to_tokens(input_ids[segment_ids==0])
    second_sent = tokenizer.convert_ids_to_tokens(input_ids[segment_ids==1])
    print(''.join(first_sent))
    print('-----SECOND TOKEN SEQUENCE-----')
    print(''.join(second_sent))

In [48]:
train_f_e = processor.convert_examples_to_features(
    train_b_e, argD.max_length, tokenizer)[0]
print('label:', train_b_e[0].label)
for i, f in enumerate(train_f_e):
    print('-----EXAMPLE{}-----'.format(i+1))
    show_baseline_features(f)

#examples 4
#features 1
label: 2
-----EXAMPLE1-----
-----FIRST TOKEN SEQUENCE-----
[CLS]在很久很久以前，乌龟和兔子之间发生了争论，它们都说自己跑得比对方快。于是，它们决定通过比赛来一决胜负。确定了路线之后它们就开始跑了起来。兔子一个箭步冲到了前面，并且一路领先。它看到乌龟被远远抛在了后面，兔子觉得，自己应该先在树下休息一会儿。于是，它在树下坐了下来，并且很快睡着了。乌龟慢慢地超过了它，并且完成了整个赛程，开心地当上了冠军。后来，兔子醒了过来，发现自己输了。[SEP]
-----SECOND TOKEN SEQUENCE-----
乌龟和兔子为什么要进行比赛?[SEP]它们之间发生了争执[SEP]
-----EXAMPLE2-----
-----FIRST TOKEN SEQUENCE-----
[CLS]在很久很久以前，乌龟和兔子之间发生了争论，它们都说自己跑得比对方快。于是，它们决定通过比赛来一决胜负。确定了路线之后它们就开始跑了起来。兔子一个箭步冲到了前面，并且一路领先。它看到乌龟被远远抛在了后面，兔子觉得，自己应该先在树下休息一会儿。于是，它在树下坐了下来，并且很快睡着了。乌龟慢慢地超过了它，并且完成了整个赛程，开心地当上了冠军。后来，兔子醒了过来，发现自己输了。[SEP]
-----SECOND TOKEN SEQUENCE-----
乌龟和兔子为什么要进行比赛?[SEP]它们的关系很不好[SEP]
-----EXAMPLE3-----
-----FIRST TOKEN SEQUENCE-----
[CLS]在很久很久以前，乌龟和兔子之间发生了争论，它们都说自己跑得比对方快。于是，它们决定通过比赛来一决胜负。确定了路线之后它们就开始跑了起来。兔子一个箭步冲到了前面，并且一路领先。它看到乌龟被远远抛在了后面，兔子觉得，自己应该先在树下休息一会儿。于是，它在树下坐了下来，并且很快睡着了。乌龟慢慢地超过了它，并且完成了整个赛程，开心地当上了冠军。后来，兔子醒了过来，发现自己输了。[SEP]
-----SECOND TOKEN SEQUENCE-----
乌龟和兔子为什么要进行比赛?[SEP]它们都认为自己跑得快[SEP]
-----EXAMPLE4-----
-----FIRS

# Infer with Baseline Model